# Hyperparameter Tuning with Optuna

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thawn/ttt-workshop-cnn/blob/main/book/pair_programming/optuna_tuning.ipynb)

Tune learning rate, weight decay, and channels on a toy training loop.

In [1]:
# Dependency management
import sys, subprocess
for p in ['optuna', 'plotly', 'scikit-learn', 'torch']:
    try:
        __import__(p if p != 'scikit-learn' else 'sklearn')
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', p, '-q'])

In [2]:
import optuna
import torch, torch.nn as nn, torch.optim as optim
from pathlib import Path

# fix random seeds for reproducibility
_ = torch.random.manual_seed(0)

In [3]:
# Data
X = torch.randn(64,1,16,16)
y = torch.randn(64,1,16,16)

# Model
class Tiny(nn.Module):
    def __init__(self, ch=8):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1,ch,3,padding=1), nn.ReLU(), nn.Conv2d(ch,1,3,padding=1))
    def forward(self,x): return self.net(x)

# Loss
criterion = nn.MSELoss()


In [4]:
# Objective function
def objective(trial):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    wd = trial.suggest_float('weight_decay', 0.0, 1e-3)
    ch = trial.suggest_categorical('channels', [4,8,16,32])
    model = Tiny(ch)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for _ in range(10):
        opt.zero_grad(); loss = criterion(model(X), y); loss.backward(); opt.step()
    return float(loss)

In [5]:
# Create study
study_name = "tiny_optuna_study"

study = optuna.create_study(
    direction="minimize", study_name=study_name, storage=f"sqlite:///{Path('data') / study_name}.db", load_if_exists=True
)

[I 2025-09-15 10:50:45,135] Using an existing study with name 'tiny_optuna_study' instead of creating a new one.


In [6]:
# Optimize the study
study.optimize(objective, n_trials=15)
print('Best:', study.best_params)

/var/folders/8s/7jr95f1d28vflnwm1cfqv7t40000gr/T/ipykernel_60404/2863418574.py:10: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  return float(loss)
[I 2025-09-15 10:50:45,616] Trial 105 finished with value: 1.006271243095398 and parameters: {'lr': 0.003235477059194364, 'weight_decay': 0.0003943571991857662, 'channels': 32}. Best is trial 32 with value: 0.996220588684082.
[I 2025-09-15 10:50:45,745] Trial 106 finished with value: 1.0000187158584595 and parameters: {'lr': 0.0027549332514569605, 'weight_decay': 0.00031531261055469425, 'channels': 32}. Best is trial 32 with value: 0.996220588684082.
[I 2025-09-15 10:50:45,875] Trial 107 finished with value: 1.009878396987915 and parameters: {'lr': 0.002420744368846494, 'weight_decay': 0.00045482561461950726, 'channe

Best: {'lr': 0.004609140937925973, 'weight_decay': 0.0008943556142438781, 'channels': 16}


In [7]:
# Plotting
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [8]:
# Plot hyperparameter importance
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [9]:
# Plot parallel coordinates
fig = optuna.visualization.plot_parallel_coordinate(study)
fig.show()

## Exercise

Use optuna to optimize the U-Net in the [U-Net notebook](https://colab.research.google.com/github/thawn/ttt-workshop-cnn/blob/main/book/exercises/unet_bbbc039_pair_programming.ipynb). Do not forget to add the dependencies and then ** restart the notebook **

## Challenge

Can you beat the best manual tuning?

Which team gets the best test score using optuna?